In [1]:
import cv2
import numpy as np


cam = cv2.VideoCapture(0)
cv2.namedWindow('frame')


def change_val(inp):
    pass

cv2.createTrackbar('val1','frame',0,255,change_val)
cv2.createTrackbar('val2','frame',0,255,change_val)
cv2.createTrackbar('val3','frame',0,255,change_val)


cv2.createTrackbar('val1h','frame',0,255,change_val)
cv2.createTrackbar('val2h','frame',0,255,change_val)
cv2.createTrackbar('val3h','frame',0,255,change_val)

cv2.setTrackbarPos("val1","frame",182)
cv2.setTrackbarPos("val2","frame",80)
cv2.setTrackbarPos("val3","frame",0)

cv2.setTrackbarPos("val1h","frame",255)
cv2.setTrackbarPos("val2h","frame",193)
cv2.setTrackbarPos("val3h","frame",134)

name = "frame"

while(1):
    retvalue, frame = cam.read()
    
    
    low_th = (cv2.getTrackbarPos("val1","frame"),cv2.getTrackbarPos("val2","frame"),cv2.getTrackbarPos("val3","frame"))
    high_th = (cv2.getTrackbarPos("val1h","frame"),cv2.getTrackbarPos("val2h","frame"),cv2.getTrackbarPos("val3h","frame"))
        
    mask = cv2.inRange(frame,low_th, high_th)
    
    hsv_image = cv2.cvtColor(frame,cv2.COLOR_RGB2HSV)
    
    kernel = np.ones((10,10),np.uint8)
    eroded = cv2.dilate(mask,kernel)
    
    
    
    
    output = cv2.connectedComponentsWithStats(eroded, 4, cv2.CV_32S)
    labels_num = output[0]
    stats = output[2]
    true_labels = []
    for label in range(labels_num):
        if(stats[label,cv2.CC_STAT_AREA] > 1000):
            true_labels.append(label)
        
    for label in true_labels[1:]:
        l_up = stats[label,cv2.CC_STAT_LEFT]
        top = stats[label,cv2.CC_STAT_TOP ]
        width = stats[label,cv2.CC_STAT_WIDTH  ]
        heith = stats[label,cv2.CC_STAT_HEIGHT  ]
        cv2.rectangle(frame, (l_up,top),(l_up + width,top +heith),(0,255,0),3)
        
    
    contours, hierarchy = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours) != 0):        
        cnt = sorted(contours, key = cv2.contourArea, reverse = True)[0]
        
        perimeter = cv2.arcLength(cnt,True)
        epsilon = 0.01*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)

        cv2.drawContours(frame, approx, -1, (255,0,0), 3)
        k = 250* 2 *25 / 14 * 12 / 20 * 24
        L = k / perimeter
        
        cv2.setWindowTitle(name,str(L))
        
    
    
    cv2.imshow("frame",frame)
    #cv2.imshow("new one", hsv_image)
    cv2.imshow("mask",eroded)
    key = cv2.waitKey(30) & 0xFF
    
    if(key == ord('q')):
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

In [1]:
#====================================
#distance measuring
#====================================

#cv2.findContours
#cv2.arcLength
#cv2.approxPolyDP

#====================================
#image warping
#====================================

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")
    
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
    # return the warped image
    return warped

#patch distance


#gradients
#cv2.filter2d



import cv2
import numpy as np


cam = cv2.VideoCapture(0)
cv2.namedWindow('frame')
cv2.namedWindow('warped')


def change_val(inp):
    pass

cv2.createTrackbar('val1','frame',0,255,change_val)
cv2.createTrackbar('val2','frame',0,255,change_val)
cv2.createTrackbar('val3','frame',0,255,change_val)


cv2.createTrackbar('val1h','frame',0,255,change_val)
cv2.createTrackbar('val2h','frame',0,255,change_val)
cv2.createTrackbar('val3h','frame',0,255,change_val)

cv2.setTrackbarPos("val1","frame",150)
cv2.setTrackbarPos("val2","frame",0)
cv2.setTrackbarPos("val3","frame",0)

cv2.setTrackbarPos("val1h","frame",255)
cv2.setTrackbarPos("val2h","frame",139)
cv2.setTrackbarPos("val3h","frame",151)

name = "frame"
prev_approx =[]
while(1):
    retvalue, frame = cam.read()
    
    
    low_th = (cv2.getTrackbarPos("val1","frame"),cv2.getTrackbarPos("val2","frame"),cv2.getTrackbarPos("val3","frame"))
    high_th = (cv2.getTrackbarPos("val1h","frame"),cv2.getTrackbarPos("val2h","frame"),cv2.getTrackbarPos("val3h","frame"))
        
    mask = cv2.inRange(frame,low_th, high_th)
    
    hsv_image = cv2.cvtColor(frame,cv2.COLOR_RGB2HSV)
    
    kernel = np.ones((10,10),np.uint8)
    eroded = cv2.dilate(mask,kernel)
    
    
    
    
    output = cv2.connectedComponentsWithStats(eroded, 4, cv2.CV_32S)
    labels_num = output[0]
    stats = output[2]
    true_labels = []
    for label in range(labels_num):
        if(stats[label,cv2.CC_STAT_AREA] > 1000):
            true_labels.append(label)
        
    for label in true_labels[1:]:
        l_up = stats[label,cv2.CC_STAT_LEFT]
        top = stats[label,cv2.CC_STAT_TOP ]
        width = stats[label,cv2.CC_STAT_WIDTH  ]
        heith = stats[label,cv2.CC_STAT_HEIGHT  ]
        cv2.rectangle(frame, (l_up,top),(l_up + width,top +heith),(0,255,0),3)
        
    
    contours, hierarchy = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours) != 0):        
        cnt = sorted(contours, key = cv2.contourArea, reverse = True)[0]
        
        perimeter = cv2.arcLength(cnt,True)
        epsilon = 0.001*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        #cv2.drawContours(frame, approx, -1, (255,0,0), 3)
        k = 250* 2 *25 / 14 * 12 / 20 * 24
        L = k / perimeter
        cv2.setWindowTitle(name,str(L))
        
        
        p = 1.0025
        counter= 0
        while(len(approx)!= 4):
            
            val = np.abs(len(approx)-4)
            
            if(len(approx) > 4):
                epsilon = epsilon* p**(val)
            if(len(approx) < 4):
                epsilon = epsilon* (1/p)**(val)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            
            if(len(approx) == 5):
                counter+=1
                
            if(counter >= 10):        
                break
        
        cv2.drawContours(frame, approx, -1, (255,0,0), 3)
        
        if (counter < 10):
            approx = np.array([approx[0][0],approx[1][0],approx[2][0],approx[3][0]])

            warped = four_point_transform(frame, approx)
            cv2.imshow("warped",warped)

        
        
    
    
    cv2.imshow("frame",frame)
    #cv2.imshow("new one", hsv_image)
    cv2.imshow("mask",eroded)
    key = cv2.waitKey(30) & 0xFF
    
    if(key == ord('q')):
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

In [3]:
a = [[[101, 154]],
 [[101, 158]],

 [[100, 159]],

 [[100, 168]],

 [[103, 168]],

 [[104, 169]],

 [[111, 169]],

 [[112, 170]],

 [[123, 170]],

 [[123, 154]]]

In [6]:
a[0][0]

[101, 154]